In [1]:
import pandas as pd

In [2]:
results_data_path = '../data/raw_data/results.csv'  # Use your preprocessed data path
df = pd.read_csv(results_data_path)

# Display the first few rows
df.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


In [3]:
df.shape

(26699, 18)

In [4]:
df = df.reindex(sorted(df.columns), axis=1)

# Display DataFrame info
df.info()

# Check the shape of the DataFrame
print(f"DataFrame shape: {df.shape}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26699 entries, 0 to 26698
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   constructorId    26699 non-null  int64  
 1   driverId         26699 non-null  int64  
 2   fastestLap       26699 non-null  object 
 3   fastestLapSpeed  26699 non-null  object 
 4   fastestLapTime   26699 non-null  object 
 5   grid             26699 non-null  int64  
 6   laps             26699 non-null  int64  
 7   milliseconds     26699 non-null  object 
 8   number           26699 non-null  object 
 9   points           26699 non-null  float64
 10  position         26699 non-null  object 
 11  positionOrder    26699 non-null  int64  
 12  positionText     26699 non-null  object 
 13  raceId           26699 non-null  int64  
 14  rank             26699 non-null  object 
 15  resultId         26699 non-null  int64  
 16  statusId         26699 non-null  int64  
 17  time        

In [5]:
irrelevant_columns = [
    'position', 'positionText', 'positionOrder', 'fastestLapTime', 'fastestLapTime', 'fastestLapSpeed', 'fastestLap', 'points', 'time', 'laps', 'milliseconds', 'rank', 'statusId'
]

df_other = df[['driverId', 'constructorId', 'raceId', 'statusId', 'points']].copy()

# Drop the columns if they exist in the DataFrame
df_dropped= df.drop(columns=[col for col in irrelevant_columns if col in df.columns])
df_noDuplicates = df_dropped.drop_duplicates()
df_noDuplicates.shape

(26699, 6)

In [6]:
df_noDuplicates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26699 entries, 0 to 26698
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   constructorId  26699 non-null  int64 
 1   driverId       26699 non-null  int64 
 2   grid           26699 non-null  int64 
 3   number         26699 non-null  object
 4   raceId         26699 non-null  int64 
 5   resultId       26699 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 1.2+ MB


### Add new features from other tables

### Add Race Info

In [7]:
races_data_path = '../data/raw_data/races.csv' 
df_races = pd.read_csv(races_data_path)

In [8]:
df_races.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1125 entries, 0 to 1124
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   raceId       1125 non-null   int64 
 1   year         1125 non-null   int64 
 2   round        1125 non-null   int64 
 3   circuitId    1125 non-null   int64 
 4   name         1125 non-null   object
 5   date         1125 non-null   object
 6   time         1125 non-null   object
 7   url          1125 non-null   object
 8   fp1_date     1125 non-null   object
 9   fp1_time     1125 non-null   object
 10  fp2_date     1125 non-null   object
 11  fp2_time     1125 non-null   object
 12  fp3_date     1125 non-null   object
 13  fp3_time     1125 non-null   object
 14  quali_date   1125 non-null   object
 15  quali_time   1125 non-null   object
 16  sprint_date  1125 non-null   object
 17  sprint_time  1125 non-null   object
dtypes: int64(4), object(14)
memory usage: 158.3+ KB


In [9]:
df_results_races = pd.merge(df_noDuplicates, df_races[['raceId', 'year','round' ,'circuitId', 'name', 'date', 'time', 'fp1_date', 'fp2_date', 'fp3_date', 'fp1_time', 'fp2_time', 'fp3_time', 'quali_time', 'quali_date']], on=['raceId'], how='left')
df_results_races = df_results_races.rename(columns={'name': 'race_name', 'date': 'race_date', 'time': 'race_time'})
df_results_races.shape

(26699, 20)

### Add Circuit Info

In [10]:
circuits_data_path = '../data/raw_data/circuits.csv' 
df_circuits = pd.read_csv(circuits_data_path)

In [11]:
df_circuits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   circuitId   77 non-null     int64  
 1   circuitRef  77 non-null     object 
 2   name        77 non-null     object 
 3   location    77 non-null     object 
 4   country     77 non-null     object 
 5   lat         77 non-null     float64
 6   lng         77 non-null     float64
 7   alt         77 non-null     int64  
 8   url         77 non-null     object 
dtypes: float64(2), int64(2), object(5)
memory usage: 5.5+ KB


In [12]:
df_results_races_circuits = pd.merge(df_results_races, df_circuits[['circuitId', 'location', 'country','lat' ,'lng', 'alt']], on=['circuitId'], how='left')
df_results_races_circuits = df_results_races_circuits.rename(columns={'location': 'race_location', 'country': 'race_country', 'lat': 'race_lat', 'lng': 'race_lng', 'alt': 'race_alt'})
df_results_races_circuits.shape

(26699, 25)

### Grid vs. Position
* Grid is the actual starting position on race day
* position is the position in at the end of qualifying, not including penalties

In [13]:
qualifying_data_path = '../data/raw_data/qualifying.csv' 
df_qualifying = pd.read_csv(qualifying_data_path)

In [14]:
df_qualifying.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10434 entries, 0 to 10433
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   qualifyId      10434 non-null  int64 
 1   raceId         10434 non-null  int64 
 2   driverId       10434 non-null  int64 
 3   constructorId  10434 non-null  int64 
 4   number         10434 non-null  int64 
 5   position       10434 non-null  int64 
 6   q1             10434 non-null  object
 7   q2             10412 non-null  object
 8   q3             10388 non-null  object
dtypes: int64(6), object(3)
memory usage: 733.8+ KB


In [15]:
df_results_races_circuits_qualifying = pd.merge(df_results_races_circuits, df_qualifying[['raceId', 'driverId','q1' ,'q2', 'q3', 'position']], on=['raceId', 'driverId'], how='left')
df_results_races_circuits_qualifying.shape

(26699, 29)

### Adding Driver Info

In [17]:
drivers_data_path = '../data/raw_data/drivers.csv'
df_drivers = pd.read_csv(drivers_data_path)

In [18]:
df_drivers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 860 entries, 0 to 859
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   driverId     860 non-null    int64 
 1   driverRef    860 non-null    object
 2   number       860 non-null    object
 3   code         860 non-null    object
 4   forename     860 non-null    object
 5   surname      860 non-null    object
 6   dob          860 non-null    object
 7   nationality  860 non-null    object
 8   url          860 non-null    object
dtypes: int64(1), object(8)
memory usage: 60.6+ KB


In [21]:
df_results_races_circuits_qualifying_drivers = pd.merge(df_results_races_circuits_qualifying, df_drivers[['driverId', 'dob','nationality']], on=['driverId'], how='left')
df_results_races_circuits = df_results_races_circuits.rename(columns={'dob': 'driver_dob', 'nationality': 'driver_nationality'})
df_results_races_circuits_qualifying_drivers.shape

(26699, 31)

In [22]:
df_results_races_circuits_qualifying

,constructorId,driverId,grid,number,raceId,resultId,year,round,circuitId,race_name,...,quali_date,race_location,race_country,race_lat,race_lng,race_alt,q1,q2,q3,position
0,1,1,1,22,18,1,2008,1,1,Australian Grand Prix,...,\N,Melbourne,Australia,-37.8497,144.9680,10,1:26.572,1:25.187,1:26.714,1.0
1,2,2,5,3,18,2,2008,1,1,Australian Grand Prix,...,\N,Melbourne,Australia,-37.8497,144.9680,10,1:25.960,1:25.518,1:27.236,5.0
2,3,3,7,7,18,3,2008,1,1,Australian Grand Prix,...,\N,Melbourne,Australia,-37.8497,144.9680,10,1:26.295,1:26.059,1:28.687,7.0
3,4,4,11,5,18,4,2008,1,1,Australian Grand Prix,...,\N,Melbourne,Australia,-37.8497,144.9680,10,1:26.907,1:26.188,\N,12.0
4,1,5,3,23,18,5,2008,1,1,Australian Grand Prix,...,\N,Melbourne,Australia,-37.8497,144.9680,10,1:25.664,1:25.452,1:27.079,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26694,6,832,0,55,1141,26700,2024,21,18,São Paulo Grand Prix,...,2024-11-02,São Paulo,Brazil,-23.7036,-46.6997,785,1:30.303,1:29.406,\N,14.0
26695,3,861,16,43,1141,26701,2024,21,18,São Paulo Grand Prix,...,2024-11-02,São Paulo,Brazil,-23.7036,-46.6997,785,1:31.270,\N,\N,18.0
26696,3,848,7,23,1141,26702,2024,21,18,São Paulo Grand Prix,...,2024-11-02,São Paulo,Brazil,-23.7036,-46.6997,785,1:29.072,1:25.889,1:24.657,7.0
26697,117,840,10,18,1141,26703,2024,21,18,São Paulo Grand Prix,...,2024-11-02,São Paulo,Brazil,-23.7036,-46.6997,785,1:30.580,1:26.334,\N,10.0


### Adding PitStop Data

In [13]:
pitstop_data_path = '../data/raw_data/pit_stops.csv'
df_pitstops = pd.read_csv(pitstop_data_path)

In [14]:
# Function to analyze pit stop durations for a specific driver in a specific race
def analyze_pit_stops(race_id, driver_id):
    # Filter the DataFrame for the specified race and driver
    driver_pit_stops = df[(df['raceId'] == race_id) & (df['driverId'] == driver_id)]

    # Check if there are any pit stops for the given criteria
    if driver_pit_stops.empty:
        print(f"No pit stop data found for driverId {driver_id} in raceId {race_id}.")
        return

    # Convert 'milliseconds' column to numeric if it's not already
    driver_pit_stops['milliseconds'] = pd.to_numeric(driver_pit_stops['milliseconds'], errors='coerce')

    # Drop rows with NaN values in 'milliseconds' column
    driver_pit_stops = driver_pit_stops.dropna(subset=['milliseconds'])

    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = driver_pit_stops['milliseconds'].quantile(0.25)
    Q3 = driver_pit_stops['milliseconds'].quantile(0.75)

    # Calculate IQR
    IQR = Q3 - Q1

    # Define the lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter out outliers
    filtered_pit_stops = driver_pit_stops[(driver_pit_stops['milliseconds'] >= lower_bound) &
                                          (driver_pit_stops['milliseconds'] <= upper_bound)]

    # Check if there are any pit stops left after filtering
    if filtered_pit_stops.empty:
        print(f"All pit stops for driverId {driver_id} in raceId {race_id} are considered outliers.")
        return

    # Calculate min, max, and average durations
    min_duration = filtered_pit_stops['milliseconds'].min()
    max_duration = filtered_pit_stops['milliseconds'].max()
    avg_duration = filtered_pit_stops['milliseconds'].mean()

In [15]:
df_pitstops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11244 entries, 0 to 11243
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   raceId        11244 non-null  int64 
 1   driverId      11244 non-null  int64 
 2   stop          11244 non-null  int64 
 3   lap           11244 non-null  int64 
 4   time          11244 non-null  object
 5   duration      11244 non-null  object
 6   milliseconds  11244 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 615.0+ KB


In [16]:
df_results_quali = pd.merge(df_noDuplicates, df_qualifying[['raceId', 'driverId', 'q1', 'q2', 'q3', 'position']],
                            on=['raceId', 'driverId'])